In [ ]:
# !pip install geemap

In [1]:
import geemap
import ee

ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas-jupyter')

## Update the geemap package

If you run into errors with this notebook, please uncomment the line below to update the [geemap](https://github.com/gee-community/geemap#installation) package to the latest version from GitHub. 
Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [3]:
# geemap.update_package()

## Create an interactive map

In [6]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
dataset_jug = ee.ImageCollection('projects/sat-io/open-datasets/GLO-30')
Map.addLayer(dataset_jug, {}, 'projects/sat-io/open-datasets/GLO-30')

In [10]:
dataset_znt = ee.Image('UK/EA/ENGLAND_1M_TERRAIN/2022')
Map.addLayer(dataset_znt, {}, 'UK/EA/ENGLAND_1M_TERRAIN/2022')

In [32]:
dataset = ee.Image('USGS/GAP/CONUS/2011')

visualization = {
    "bands": ['landcover'],
    "min": 1.0,
    "max": 584.0,
}

Map.setCenter(-98.58, 38.14, 4)

Map.addLayer(dataset, visualization, 'GAP CONUS')

In [12]:
#*
# Function to mask clouds using the Sentinel-2 QA band
# @param {ee.Image} image Sentinel-2 image
# @return {ee.Image} cloud masked Sentinel-2 image
#
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
.filterDate('2020-01-01', '2020-01-30') \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
.map(maskS2clouds)

visualization = {
    "min": 0.0,
    "max": 0.3,
    "bands": ['B4', 'B3', 'B2'],
}

Map.setCenter(83.277, 17.7009, 12)

Map.addLayer(dataset.mean(), visualization, 'RGB')

In [34]:
dataset_evj = ee.FeatureCollection('projects/sat-io/open-datasets/global_fertilizer_use_centroid')
Map.addLayer(dataset_evj, {}, 'projects/sat-io/open-datasets/global_fertilizer_use_centroid')

In [36]:
dataset_ypv = ee.FeatureCollection('ESA/WorldCereal/AEZ/v100')
Map.addLayer(dataset_ypv, {}, 'ESA/WorldCereal/AEZ/v100')

## Convert marker to ee.Geometry

In [44]:
Map.search_locations

[[Egypt],
 [Egypt, Stroud, Gloucestershire, England],
 [Egypt, Buckinghamshire, England],
 [Egypt, Alabama],
 [Egypt, Arkansas],
 [Egypt, Pennsylvania],
 [Egypt, South Hill, Callington, Cornwall, England],
 [Egypt, Winchester, Hampshire, England]]

In [58]:
Map.search_loc_geom

In [60]:
location = Map.search_loc_geom
print(location.getInfo())

AttributeError: 'NoneType' object has no attribute 'getInfo'